In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go

In [ ]:
exp_data1=pd.DataFrame({
'SO4':[3.00,2.51,1.84,1.42,1.16,1.05],
'FA':[43.9,36,28,21,15,9.9],
'DCP':[0.0,1.61,3.26,4.94,6.66,8.42]})
exp_data1

In [ ]:
exp_data2=pd.DataFrame({
'SO4':[3.04,2.23,1.61,1.15,0.93,0.84],
'FA':[50.0,41,33,25,18,12],
'DCP':[0.0,1.62,3.27,4.96,6.69,8.45]})
exp_data2

In [ ]:
exp_data3=pd.DataFrame({
'SO4':[2.97,2.19,1.56,1.09,0.85,0.76],
'FA':[55,46,38,29,22,15],
'DCP':[0.0,1.62,3.29,5.00,6.71,8.49]})
exp_data3

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=exp_data1.SO4,y=exp_data1.FA,z=exp_data1.DCP, mode='markers' ))
fig.add_trace(go.Scatter3d(x=exp_data2.SO4,y=exp_data2.FA,z=exp_data2.DCP, mode='markers' ))
fig.add_trace(go.Scatter3d(x=exp_data3.SO4,y=exp_data3.FA,z=exp_data3.DCP, mode='markers' ))


fig.update_layout(scene = dict(
                    xaxis_title='SO4',
                    yaxis_title='FA',
                    zaxis_title='DCP'),
                    width=700,
                    margin=dict(r=20, b=50, l=10, t=10))

X, Y, Z = np.mgrid[:1:20j, :1:20j, :1:20j]
vol = (X - 1)**2 + (Y - 1)**2 + Z**2


fig.add_trace(go.Volume(
    x=X.flatten(), y=Y.flatten(), z=Z.flatten(),
    value=vol.flatten(),
    isomin=0.2,
    isomax=0.7,
    opacity=0.2,
    surface_count=21,
    caps= dict(x_show=True, y_show=True, z_show=True, x_fill=1), # with caps (default mode)
    ))

fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=4, range=[0,3.2],),
        yaxis = dict(nticks=4, range=[0,55],),
        zaxis = dict(nticks=4, range=[-.5,9],),),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10))
fig.show()

### filling each experiment

In [ ]:
# exp_data1

In [ ]:
def fill_each_line(df, l):
    n=l+1 #l=0 ==> no addition
    col=df.columns
    new_df=pd.DataFrame(columns=col)
    for i in range(len(df)-1):
        row1=df.iloc[i]
        row2=df.iloc[i+1]
        for m in range(n):
            new_row=pd.DataFrame({
                      'SO4':[row1.SO4-m*(row1.SO4-row2.SO4)/n],
                      'FA':[row1.FA-m*(row1.FA-row2.FA)/n],
                      'DCP':[row1.DCP-m*(row1.DCP-row2.DCP)/n],
            })
            new_df=pd.concat([new_df,new_row])
    new_df=pd.concat([new_df,pd.DataFrame([df.iloc[-1]])])
    new_df.reset_index(inplace=True, drop=True)
    return new_df

In [ ]:
n=2
exp_filled_data1=fill_each_line(exp_data1, n)
exp_filled_data2=fill_each_line(exp_data2, n)
exp_filled_data3=fill_each_line(exp_data3, n)
# exp_filled_data1

### filling data between FA

In [ ]:
def fill_between_lines(df1,df2,n):
    set=[]
    for m in range(n):
        new_df=pd.DataFrame()
        for i in range(len(df1)):
            row1,row2=df1.iloc[i],df2.iloc[i]
            new_row=pd.DataFrame({
                      'SO4':[(row1[0]-(m+1)*(row1[0]-row2[0])/(n+1))],
                      'FA':[(row1[1]-(m+1)*(row1[1]-row2[1])/(n+1))],
                      'DCP':[(row1[2]-(m+1)*(row1[2]-row2[2])/(n+1))]})
            new_df=pd.concat([new_df,new_row])
        new_df.reset_index(inplace=True, drop=True)
        set.append(new_df)
    return set

In [ ]:
m=2
set12=fill_between_lines(exp_filled_data1,exp_filled_data2,m)
set23=fill_between_lines(exp_filled_data2,exp_filled_data3,m)
# set12

### combining exp datas

In [ ]:
full_set=[exp_filled_data1]+set12+[exp_filled_data2]+set23+[exp_filled_data3]
# full_set

In [ ]:
fig = go.Figure()

for i in range(len(full_set)):
    df=full_set[i]
    x=df.SO4
    y=df.FA
    z=df.DCP

    fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers+lines', line_color=z, marker_size=3, opacity=0.8))

fig.update_layout(scene = dict(
                    xaxis_title='SO4',
                    yaxis_title='FA',
                    zaxis_title='DCP'),
                    width=700,
                    margin=dict(r=20, b=50, l=10, t=10))
# fig.update_scenes(zaxis_autorange="reversed")
# fig.write_html("traces.html")
fig.show()

### converting exp data to model rows

In [ ]:
# exp_data1

In [ ]:
def exp_to_model(df):
    col=['SO4','FA','DCP','y_SO4','y_FA']
    new_df=pd.DataFrame(columns=col)
#     DCP=0
    for n in range(len(df)):
        DCP=df.iloc[n].DCP
        for m in range(n,len(df)):
            row1,row2=df.iloc[n],df.iloc[m]
            new_row=pd.DataFrame({
                      col[0]:[row1[0]],
                      col[1]:[row1[1]],
                      col[2]:[row2[2]-DCP], #תוספת DCP בלבד
                      col[3]:[row2[0]], #ערכים שאחרי תוספת DCP
                      col[4]:[row2[1]]})
            new_df=pd.concat([new_df,new_row])
#         DCP=new_df.iloc[n]['DCP']
#         print (DCP)
    new_df.reset_index(inplace=True, drop=True)
    return new_df            

In [ ]:
# exp_to_model(exp_data1)

In [ ]:
full_set_model=[exp_to_model(df) for df in full_set]
len(full_set_model)

### vizualizing the model rows

In [ ]:
l=len(full_set_model[0])
set=[]
set_len=1
i=0
while i <l:
    new_set=[]
    for j in range(set_len):
        i+=1
        new_set.append(l-i)
    set.append(new_set)
    set_len+=1
set

In [ ]:
fig = go.Figure()

for i in range(len(full_set_model)):
    df=full_set_model[i]
    for j in range(len(set)):
        x=df.y_SO4[set[j]]
        y=df.y_FA[set[j]]
        z=df.DCP[set[j]]
        fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers+lines', line_color=z, marker_size=3, opacity=0.4))

fig.update_layout(scene = dict(
                    xaxis_title='SO4',
                    yaxis_title='FA',
                    zaxis_title='DCP'),
                    width=700,
                    margin=dict(r=20, b=50, l=10, t=10))

# fig.write_html("model_rows-2_2.html")
fig.show()

### writing csv

In [ ]:
joined_full_model=pd.DataFrame()
for i in range(len(full_set_model)):
    joined_full_model=pd.concat([joined_full_model,full_set_model[i]])

In [ ]:
# joined_full_model.to_csv('joined_full_model-2_2.csv')